In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Načítavanie dát

In [9]:
data = pd.read_csv('data/data_phase_1.csv')

# Podiel práce

Tulach 50%

Černega 50%

# Riešenie

## 2.1 A

Na konci prvej fázy zadania sme si exportli mergnutý dataset všetkých 4 tabuliek a ten sme následne importovali sem:

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45586 entries, 0 to 45585
Data columns (total 49 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ts                          45586 non-null  object 
 1   imei                        45586 non-null  int64  
 2   mwra                        45586 non-null  float64
 3   c.android.youtube           45586 non-null  float64
 4   c.android.chrome            45586 non-null  float64
 5   c.android.gm                45586 non-null  float64
 6   c.dogalize                  45586 non-null  float64
 7   c.katana                    45586 non-null  float64
 8   c.UCMobile.x86              45586 non-null  float64
 9   c.updateassist              45586 non-null  float64
 10  c.android.vending           45586 non-null  float64
 11  c.UCMobile.intl             45586 non-null  float64
 12  c.raider                    45586 non-null  float64
 13  p.android.packageinstaller  455

Moje zdôvodnenie pre tento krok je ten, že sme v prvej fáze niektoré stĺpce niektorých tabuliek mazali a prerábali na dva osobitné stĺpce (konkrétne stĺpec location sme úplne zmazali a tie dáta sme prerobili na latitude a longtitude, keďže je lepšie mať univerzálny formát dát). Sekvenčne je vytvorenie testovacej a trénovacej sady z týchto dát ako prvá úloha v tejto fáze zadania, a síce už máme odstránených outlierov, mohli nám vzniknúť nové, ktoré by sme dokázali odstrániť pomocou Pipeline tak či tak. Vytvoríme si teda testovaciu a trénovaciu sadu:

In [11]:
# Define the target variable and features
X = data.drop(columns=['mwra'])
y = data['mwra']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (36468, 48)
X_test shape: (9118, 48)
y_train shape: (36468,)
y_test shape: (9118,)


Ďalej budeme už robiť zatiaľ s trénovacím datasetom.

## 2.1 B

### Imputing

Najprv vytvoríme nový dataframe, kde odstránime riadky s nedefinovanými hodnotami z nášho daného datasetu:

In [12]:
# Remove NA values from the training dataset
X_train_cleaned = X_train.dropna()
y_train_cleaned = y_train[X_train_cleaned.index]

# Print the number of rows in the cleaned training dataset
print(f"Number of rows in the cleaned training dataset: {X_train_cleaned.shape[0]}")

Number of rows in the cleaned training dataset: 7779


Vidíme, že z pôvodných 36468 hodnôt sa nám počet zredukoval na 7779, preto je to celkom nešťastné riešenie a bolo by lepšie, keby sme chýbajúce hodnoty nejakou štatistickou metódou do nášho datasetu doplnili.

Tu teda doplníme hodnoty tak, že pre každý atribút sa zoberie priemerná hodnota, resp. najčastejšia hodnota pre kategorické atribúty a doplní sa do jednotlivých atribútov pomocou triedy SimpleImputer:

In [13]:
# Identify numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create imputers
imputer_num = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')

# Fit and transform the data
X_train[numerical_cols] = imputer_num.fit_transform(X_train[numerical_cols])
X_train[categorical_cols] = imputer_cat.fit_transform(X_train[categorical_cols])

# Check if every column has the same amount of non-null values
non_null_counts = X_train.notnull().sum()
print(non_null_counts)
all_equal = non_null_counts.nunique() == 1
print(f"All columns have the same amount of non-null values: {all_equal}")

ts                            36468
imei                          36468
c.android.youtube             36468
c.android.chrome              36468
c.android.gm                  36468
c.dogalize                    36468
c.katana                      36468
c.UCMobile.x86                36468
c.updateassist                36468
c.android.vending             36468
c.UCMobile.intl               36468
c.raider                      36468
p.android.packageinstaller    36468
p.android.documentsui         36468
p.system                      36468
p.android.externalstorage     36468
p.android.chrome              36468
p.android.settings            36468
p.android.gm                  36468
p.inputmethod.latin           36468
p.android.vending             36468
p.process.gapps               36468
p.notifier                    36468
p.katana                      36468
p.google                      36468
p.android.defcontainer        36468
p.simulator                   36468
p.android.gms               

Vidíme, že každý stĺpec má rovnaký počet hodnôt, ktoré sú nenulové, to znamená, že sa úspešne nahradili nulové hodnoty v datasete.

### 